In [1]:
%load_ext nb_black
import pandas as pd
import numpy as np
import regex as re

<IPython.core.display.Javascript object>

In [2]:
data = "datasets/challenge_dataset"
immo_eliza = pd.read_table(data, sep=",")
df = immo_eliza[["locality", "postcode", "kitchen_has", "furnished", "facades_number"]]
immo_eliza.shape

/home/sara/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (11,12,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(93068, 22)

<IPython.core.display.Javascript object>

# Functions

Functions here clean the following columns:

- **postcode**
- **furnished**
- **kitchen_has**
- **facades_number**

Modifications on the **locality** column will be performed once the data of the 4 members of the group is merged.

In [3]:
def postcodes_fun():
    "This function extracts postcodes from locality and adds them into null postcodes"

    # Extract postcodes from locality and add them to postcode column
    df["loc_code"] = df["locality"].str.extract(r"(\b[0-9]{4}\b)")
    df.loc[df.postcode.isnull(), "postcode"] = df.loc_code

    # Fill null vales with '-999'
    df["postcode"].fillna("-999", inplace=True)

    # Delete dummy column
    del df["loc_code"]

    # Return column
    return df["postcode"]


def bool_to_bin(x, final_type):
    """This function converts a True/False column into a 0/1"""

    df[x] = df[x].astype(str)
    df[x] = df[x].map({"False": 0, "True": 1, "nan": -999})
    df[x].fillna(-999, inplace=True)
    df[x] = df[x].astype(final_type)
    return df[x]


def facades_num():
    """This function cleans facades_number by covnerting values into integers
    and assign '0', 'nan' and 'none' to -999 (=treat as null)"""

    df["facades_number"] = df["facades_number"].apply(str)
    df.facades_number.replace("(\.0)", "", regex=True, inplace=True)
    df["facades_number"] = df.facades_number.replace(
        ["nan", "None", "0"], -999
    )  # maybe treat 0 as -999 too?
    df["facades_number"] = df["facades_number"].astype(int)
    return df["facades_number"]

<IPython.core.display.Javascript object>

### Calling functions

In [4]:
df["postcode"] = postcodes_fun()
df["furnished"] = bool_to_bin("furnished", int)
df["kitchen_has"] = bool_to_bin("kitchen_has", int)
df["facades_number"] = facades_num()

/home/sara/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/sara/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/sara/.local/lib/python3.6/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<IPython.core.display.Javascript object>

### Save to csv file

In [5]:
df.to_csv("loc_pc_furn_kitch-fac.csv")

<IPython.core.display.Javascript object>

## Locality: to be executed AFTER merging

As locality values are not accurate in our dataset, we pick (cleaned) postal codes as reference. Now we can assign an accurate locality value to these postal codes by merging the file with an external dataset with all Belgian postcodes. This allows us to also make sure that no postal codes other than Belgian are in our dataset. 

In [6]:
data_final = pd.read_csv("final_no_dupes.csv")
data_final

/home/sara/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,kitchen_has,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
0,0,4180,4180.0,1,mixed_use_building,295000,-999,3,242.0,1,...,0,1.0,36.0,1.0,1000,1403.0,1403.0,-999,0,good
1,1,8730,8730.0,1,villa,675000,-999,4,349.0,1,...,0,0.0,-999.0,1.0,977,1526.0,1526.0,-999,0,as_new
2,2,4020,4020.0,1,apartment_block,250000,-999,5,303.0,1,...,0,0.0,-999.0,0.0,0,760.0,760.0,-999,0,to_renovate
3,3,1200,1200.0,1,house,545000,-999,4,235.0,1,...,0,0.0,-999.0,0.0,0,63.0,63.0,-999,0,just_renovated
4,4,1190,1190.0,1,mixed_use_building,500000,-999,2,220.0,1,...,0,0.0,-999.0,1.0,60,193.0,193.0,-999,0,as_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67235,67235,2650,2650.0,0,-999,210000,-999,2,-999.0,1,...,0,-999.0,-999.0,0.0,-999,-999.0,-999.0,2,0,to_renovate
67236,67236,3700,3700.0,0,-999,239000,-999,1,-999.0,1,...,0,-999.0,-999.0,0.0,-999,-999.0,-999.0,4,0,good
67237,67237,3800,3800.0,0,-999,259000,-999,2,-999.0,1,...,0,-999.0,-999.0,0.0,-999,-999.0,-999.0,2,0,as_new
67238,67238,2610,2610.0,0,-999,254000,-999,2,-999.0,1,...,0,-999.0,-999.0,0.0,-999,-999.0,-999.0,2,0,-999


<IPython.core.display.Javascript object>

In [7]:
# Replace -999 by Nan

data_final.replace(["-999", -999], np.nan, inplace=True)

# Convert columns that need to be int type to Int64Dtype (and postcode to object)

data_final["rooms_number"] = pd.Series(
    data_final["rooms_number"], dtype=pd.Int64Dtype()
)
data_final["kitchen_has"] = pd.Series(data_final["kitchen_has"], dtype=pd.Int64Dtype())
data_final["furnished"] = pd.Series(data_final["furnished"], dtype=pd.Int64Dtype())
data_final["open_fire"] = pd.Series(data_final["open_fire"], dtype=pd.Int64Dtype())
data_final["terrace"] = pd.Series(data_final["terrace"], dtype=pd.Int64Dtype())
data_final["garden"] = pd.Series(data_final["garden"], dtype=pd.Int64Dtype())
data_final["garden_area"] = pd.Series(data_final["garden_area"], dtype=pd.Int64Dtype())
data_final["facades_number"] = pd.Series(
    data_final["facades_number"], dtype=pd.Int64Dtype()
)
data_final["swimming_pool_has"] = pd.Series(
    data_final["swimming_pool_has"], dtype=pd.Int64Dtype()
)
data_final["postcode"] = pd.Series(data_final["postcode"], dtype=pd.Int64Dtype())

<IPython.core.display.Javascript object>

In [8]:
data_final

,Unnamed: 0,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,kitchen_has,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
0,0,4180,4180,1,mixed_use_building,295000.0,NaN,3,242.0,1,...,0,1,36.0,1,1000,1403.0,1403.0,<NA>,0,good
1,1,8730,8730,1,villa,675000.0,NaN,4,349.0,1,...,0,0,NaN,1,977,1526.0,1526.0,<NA>,0,as_new
2,2,4020,4020,1,apartment_block,250000.0,NaN,5,303.0,1,...,0,0,NaN,0,0,760.0,760.0,<NA>,0,to_renovate
3,3,1200,1200,1,house,545000.0,NaN,4,235.0,1,...,0,0,NaN,0,0,63.0,63.0,<NA>,0,just_renovated
4,4,1190,1190,1,mixed_use_building,500000.0,NaN,2,220.0,1,...,0,0,NaN,1,60,193.0,193.0,<NA>,0,as_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67235,67235,2650,2650,0,NaN,210000.0,NaN,2,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,2,0,to_renovate
67236,67236,3700,3700,0,NaN,239000.0,NaN,1,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,4,0,good
67237,67237,3800,3800,0,NaN,259000.0,NaN,2,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,2,0,as_new
67238,67238,2610,2610,0,NaN,254000.0,NaN,2,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,2,0,NaN


<IPython.core.display.Javascript object>

In [9]:
data_final.dtypes

Unnamed: 0             int64
locality              object
postcode               Int64
house_is               int64
property_subtype      object
price                float64
sale                  object
rooms_number           Int64
area                 float64
kitchen_has            Int64
furnished              Int64
open_fire              Int64
terrace                Int64
terrace_area         float64
garden                 Int64
garden_area            Int64
land_surface         float64
land_plot_surface    float64
facades_number         Int64
swimming_pool_has      Int64
building_state        object
dtype: object

<IPython.core.display.Javascript object>

In [10]:
# 1. Assign postcode column as the dataframe index
data_final.set_index(data_final["postcode"], inplace=True)
data_final

,Unnamed: 0,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,kitchen_has,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
postcode,,,,,,,,,,,,,,,,,,,,,
4180,0,4180,4180,1,mixed_use_building,295000.0,NaN,3,242.0,1,...,0,1,36.0,1,1000,1403.0,1403.0,<NA>,0,good
8730,1,8730,8730,1,villa,675000.0,NaN,4,349.0,1,...,0,0,NaN,1,977,1526.0,1526.0,<NA>,0,as_new
4020,2,4020,4020,1,apartment_block,250000.0,NaN,5,303.0,1,...,0,0,NaN,0,0,760.0,760.0,<NA>,0,to_renovate
1200,3,1200,1200,1,house,545000.0,NaN,4,235.0,1,...,0,0,NaN,0,0,63.0,63.0,<NA>,0,just_renovated
1190,4,1190,1190,1,mixed_use_building,500000.0,NaN,2,220.0,1,...,0,0,NaN,1,60,193.0,193.0,<NA>,0,as_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,67235,2650,2650,0,NaN,210000.0,NaN,2,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,2,0,to_renovate
3700,67236,3700,3700,0,NaN,239000.0,NaN,1,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,4,0,good
3800,67237,3800,3800,0,NaN,259000.0,NaN,2,NaN,1,...,0,<NA>,NaN,0,<NA>,NaN,NaN,2,0,as_new


<IPython.core.display.Javascript object>

In [11]:
# 2. Load external dataset of belgian postal codes
data_postcodes = "datasets/be_postal_codes.csv"
loc_pc = pd.read_csv(data_postcodes, encoding="latin")
loc_pc

,Postal Code,Place Name,State,State Abbreviation,City,Latitude,Longitude
0,1000,Bruxelles,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
1,1005,Conseil Region Bruxelles-Capitale,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
2,1006,Raad Vlaamse Gemeenschapscommissie,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
3,1007,Ass. Commiss. Communau. française,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
4,1008,Chambre des Représentants,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
...,...,...,...,...,...,...,...
1170,7972,Beloeil Ellignies-Sainte-Anne,Wallonie,WAL,Hainaut,50.5618,3.6727
1171,7973,Beloeil Grandglise,Wallonie,WAL,Hainaut,50.5057,3.6963
1172,1100,Postcheque,NaN,NaN,NaN,50.8350,4.3647
1173,1105,SOC,NaN,NaN,NaN,50.8350,4.3647


<IPython.core.display.Javascript object>

In [12]:
# 3. Merge files by matching index of left file with "Postal Code" column of right file
merged_df = data_final.merge(loc_pc, left_index=True, right_on="Postal Code")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64570 entries, 637 to 40
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          64570 non-null  int64  
 1   locality            64568 non-null  object 
 2   postcode            64570 non-null  Int64  
 3   house_is            64570 non-null  int64  
 4   property_subtype    58713 non-null  object 
 5   price               43342 non-null  float64
 6   sale                10803 non-null  object 
 7   rooms_number        63523 non-null  Int64  
 8   area                53718 non-null  float64
 9   kitchen_has         54828 non-null  Int64  
 10  furnished           52591 non-null  Int64  
 11  open_fire           58938 non-null  Int64  
 12  terrace             48949 non-null  Int64  
 13  terrace_area        15757 non-null  float64
 14  garden              64570 non-null  Int64  
 15  garden_area         36907 non-null  Int64  
 16  land_

<IPython.core.display.Javascript object>

In [13]:
# 4. Reset index of this new merged file
merged_df = merged_df.reset_index()

<IPython.core.display.Javascript object>

In [24]:
merged_df

,index,Unnamed: 0,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,facades_number,swimming_pool_has,building_state,Postal Code,Place Name,State,State Abbreviation,City,Latitude,Longitude
0,637,0,4180,4180,1,mixed_use_building,295000.0,NaN,3,242.0,...,<NA>,0,good,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
1,637,386,4180,4180,1,mixed_use_building,229000.0,NaN,4,312.0,...,<NA>,0,to_renovate,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
2,637,1915,4180,4180,1,apartment_block,150000.0,NaN,6,450.0,...,<NA>,0,to_renovate,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
3,637,3234,4180,4180,1,house,195000.0,NaN,3,242.0,...,<NA>,0,good,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
4,637,3368,4180,4180,1,villa,419000.0,NaN,6,192.0,...,<NA>,0,good,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64565,361,56820,8421,8421,1,house,NaN,NaN,5,550.0,...,3,<NA>,to_renovate,8421,De Haan,Vlaanderen,VLG,West-Vlaanderen,51.2667,3.0333
64566,361,64626,8421,8421,1,NaN,495000.0,NaN,5,NaN,...,3,0,to_renovate,8421,De Haan,Vlaanderen,VLG,West-Vlaanderen,51.2667,3.0333
64567,899,61347,6441,6441,0,apartment,NaN,NaN,3,143.0,...,<NA>,<NA>,as_new,6441,Froidchapelle Erpion,Wallonie,WAL,Hainaut,50.0574,4.3157
64568,15,64744,1044,1044,1,NaN,589000.0,NaN,5,NaN,...,4,0,good,1044,RTBF,Bruxelles-Capitale,BRU,Bruxelles,50.8412,4.3704


<IPython.core.display.Javascript object>

In [25]:
# 5. Create new data file containing only the columns we are interested in (in the right order)
complete_data = merged_df[
    merged_df.columns[
        [
            23,
            26,
            3,
            24,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            19,
            20,
            21,
        ]
    ]
]
complete_data

,Place Name,City,postcode,State,house_is,property_subtype,price,sale,rooms_number,area,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
0,Hamoir Comblain-Fairon,Liège,4180,Wallonie,1,mixed_use_building,295000.0,NaN,3,242.0,...,0,1,36.0,1,1000,1403.0,1403.0,<NA>,0,good
1,Hamoir Comblain-Fairon,Liège,4180,Wallonie,1,mixed_use_building,229000.0,NaN,4,312.0,...,0,1,20.0,0,0,9732.0,9732.0,<NA>,0,to_renovate
2,Hamoir Comblain-Fairon,Liège,4180,Wallonie,1,apartment_block,150000.0,NaN,6,450.0,...,0,1,29.0,0,0,150.0,150.0,<NA>,0,to_renovate
3,Hamoir Comblain-Fairon,Liège,4180,Wallonie,1,house,195000.0,NaN,3,242.0,...,0,1,36.0,1,100,475.0,475.0,<NA>,0,good
4,Hamoir Comblain-Fairon,Liège,4180,Wallonie,1,villa,419000.0,NaN,6,192.0,...,0,1,50.0,0,0,8792.0,8792.0,<NA>,0,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64565,De Haan,West-Vlaanderen,8421,Vlaanderen,1,house,NaN,NaN,5,550.0,...,0,0,NaN,1,<NA>,NaN,NaN,3,<NA>,to_renovate
64566,De Haan,West-Vlaanderen,8421,Vlaanderen,1,NaN,495000.0,NaN,5,NaN,...,1,<NA>,NaN,1,<NA>,NaN,NaN,3,0,to_renovate
64567,Froidchapelle Erpion,Hainaut,6441,Wallonie,0,apartment,NaN,NaN,3,143.0,...,0,0,NaN,0,0,NaN,NaN,<NA>,<NA>,as_new
64568,RTBF,Bruxelles,1044,Bruxelles-Capitale,1,NaN,589000.0,NaN,5,NaN,...,0,<NA>,NaN,1,<NA>,NaN,NaN,4,0,good


<IPython.core.display.Javascript object>

In [28]:
complete_data.rename(
    columns={"City": "city", "State": "region", "Place Name": "locality"}, inplace=True
)

/home/sara/.local/lib/python3.6/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


<IPython.core.display.Javascript object>

In [35]:
complete_data.isna().sum()

locality                 0
city                     0
postcode                 0
region                   0
house_is                 0
property_subtype      5857
price                21228
sale                 53767
rooms_number          1047
area                 10852
kitchen_has           9742
furnished            11979
open_fire             5632
terrace              15621
terrace_area         48813
garden                   0
garden_area          27663
land_surface         33760
land_plot_surface    33462
facades_number       31730
swimming_pool_has    16225
building_state       29525
dtype: int64

<IPython.core.display.Javascript object>

In [31]:
complete_data.to_csv("def_dataset.csv", encoding="utf8")

<IPython.core.display.Javascript object>

In [ ]:
# Dataset with complete cases of price

In [55]:
df_price = pd.read_csv("def_dataset.csv")
# df_price["price"] = np.isfinite(df_price["price"])
df_price = df_price.loc[pd.notnull(df_price.price), :]

<IPython.core.display.Javascript object>

In [58]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43342 entries, 0 to 64569
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         43342 non-null  int64  
 1   locality           43342 non-null  object 
 2   city               43342 non-null  object 
 3   postcode           43342 non-null  int64  
 4   region             43342 non-null  object 
 5   house_is           43342 non-null  int64  
 6   property_subtype   37485 non-null  object 
 7   price              43342 non-null  float64
 8   sale               185 non-null    object 
 9   rooms_number       43089 non-null  float64
 10  area               35047 non-null  float64
 11  kitchen_has        40418 non-null  float64
 12  furnished          37485 non-null  float64
 13  open_fire          43341 non-null  float64
 14  terrace            37485 non-null  float64
 15  terrace_area       15103 non-null  float64
 16  garden             433

<IPython.core.display.Javascript object>

In [59]:
# Convert columns that need to be int type to Int64Dtype (and postcode to object)

df_price["rooms_number"] = pd.Series(df_price["rooms_number"], dtype=pd.Int64Dtype())
df_price["kitchen_has"] = pd.Series(df_price["kitchen_has"], dtype=pd.Int64Dtype())
df_price["furnished"] = pd.Series(df_price["furnished"], dtype=pd.Int64Dtype())
df_price["open_fire"] = pd.Series(df_price["open_fire"], dtype=pd.Int64Dtype())
df_price["terrace"] = pd.Series(df_price["terrace"], dtype=pd.Int64Dtype())
df_price["garden"] = pd.Series(df_price["garden"], dtype=pd.Int64Dtype())
df_price["garden_area"] = pd.Series(df_price["garden_area"], dtype=pd.Int64Dtype())
df_price["facades_number"] = pd.Series(
    df_price["facades_number"], dtype=pd.Int64Dtype()
)
df_price["swimming_pool_has"] = pd.Series(
    df_price["swimming_pool_has"], dtype=pd.Int64Dtype()
)
df_price["postcode"] = pd.Series(df_price["postcode"], dtype=pd.Int64Dtype())

<IPython.core.display.Javascript object>

In [60]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43342 entries, 0 to 64569
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         43342 non-null  int64  
 1   locality           43342 non-null  object 
 2   city               43342 non-null  object 
 3   postcode           43342 non-null  Int64  
 4   region             43342 non-null  object 
 5   house_is           43342 non-null  int64  
 6   property_subtype   37485 non-null  object 
 7   price              43342 non-null  float64
 8   sale               185 non-null    object 
 9   rooms_number       43089 non-null  Int64  
 10  area               35047 non-null  float64
 11  kitchen_has        40418 non-null  Int64  
 12  furnished          37485 non-null  Int64  
 13  open_fire          43341 non-null  Int64  
 14  terrace            37485 non-null  Int64  
 15  terrace_area       15103 non-null  float64
 16  garden             433

<IPython.core.display.Javascript object>

In [61]:
df_price.to_csv("price_nonnull.csv")

<IPython.core.display.Javascript object>